In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import numpy as np
from functools import partial
from scipy.optimize import minimize

import torch
import torch.optim as optim
from torch.optim import Adam

from nflows.flows import MaskedAutoregressiveFlow
from matplotlib import cm, ticker
from numpy import ma

In [ ]:
from counterfactuals.utils import plot_distributions

In [ ]:
x, y = datasets.make_moons(512, noise=0.1)
plt.scatter(x[:, 0], x[:, 1], c=y);

# Train generative model

In [ ]:
flow = MaskedAutoregressiveFlow(features=2, hidden_features=4, context_features=1)
optimizer = optim.Adam(flow.parameters())

num_iter = 10000
for i in range(num_iter):
    x, y = datasets.make_moons(128, noise=.1)
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=x, context=y).mean()
    loss.backward()
    optimizer.step()
    if (i + 1) % 1000 == 0:
        print(loss.item())

# Generative model only approach
$$ p(\mathbf{x}|y)=\gamma $$
$$ \min_{\mathbf{x}'} d(\mathbf{x}, \mathbf{x}') + \lambda \cdot \max (\max(p(\mathbf{x}'|y)+\beta, \gamma) - p(\mathbf{x}'|y'), 0) $$

In [ ]:
x_origin = torch.tensor([[1.,  0.]], requires_grad=False)
x = torch.tensor([[1.,  0.]], requires_grad=True)

def optim_function(x, x_origin, model, alpha=20.0, beta=0.01):
    model.eval()
    dist = torch.linalg.norm(x_origin-x, axis=1)

    y_orig = torch.zeros(x.shape[0]).reshape(-1, 1)
    y_hat = torch.ones(x.shape[0]).reshape(-1, 1)

    p_x_param_c_orig = model.log_prob(x, context=y_orig).exp()
    p_x_param_c_target = model.log_prob(x, context=y_hat).exp()
    p_x_orig_c_orig = model.log_prob(x_origin, context=torch.zeros(x_origin.shape[0]).reshape(-1, 1)).exp()

    p_x_param_c_orig_with_beta = p_x_param_c_orig + beta
    max_inner = torch.clamp(p_x_param_c_orig_with_beta, min=p_x_orig_c_orig)
    max_outer = torch.clamp(max_inner - p_x_param_c_target, min=torch.Tensor([0.0]))
    loss = dist + alpha * max_outer
    return loss

In [ ]:
optimizer = Adam([x], lr=0.03)
alpha = 20

num_iterations = 100
for i in range(num_iterations):
    y = optim_function(x, x_origin, flow, alpha=alpha)
    optimizer.zero_grad()
    y.backward()
    optimizer.step()
    if (i+1) % 10 == 0:
        plot_distributions(x, x_origin, flow, optim_function, alpha=alpha)